In [82]:
import json
import requests
from bs4 import BeautifulSoup

import os

In [26]:
def docx_html_url(_id): 
    return f'https://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id={_id}'

In [42]:
def articles_url(_number, _violation): 
    violation = 'nonviolation'
    if(_violation): violation = violation[3:]
        
    return f'https://hudoc.echr.coe.int/app/query/results?query=contentsitename%3AECHR%20AND%20(NOT%20(doctype%3DPR%20OR%20doctype%3DHFCOMOLD%20OR%20doctype%3DHECOMOLD))%20AND%20((languageisocode%3D%22ENG%22))%20AND%20((documentcollectionid%3D%22GRANDCHAMBER%22)%20OR%20(documentcollectionid%3D%22CHAMBER%22))%20AND%20(({violation}%3D{_number}))&select=sharepointid,Rank,ECHRRanking,languagenumber,itemid,docname,doctype,application,appno,conclusion,importance,originatingbody,typedescription,kpdate,kpdateAsText,documentcollectionid,documentcollectionid2,languageisocode,extractedappno,isplaceholder,doctypebranch,respondent,ecli,appnoparts,sclappnos&sort=&start=20&length=100&rankingModelId=11111111-0000-0000-0000-000000000000'

In [91]:
def articles_json(_number, _violation):
    results = []
    violation = 'nv'
    if(_violation): violation = violation[1:] 
    
    for result in json_data['results']:
        results.append({'id': result["columns"]["itemid"], 'name': result["columns"]["docname"], 'violation': violation})
    
    return results

In [79]:
def article_get(_id):
    url = docx_html_url(_id)
    
    r = requests.get(url).text
    soup = BeautifulSoup(r)
    
    for script in soup(["script", "style"]):
        script.extract()
        
    text = soup.get_text()
    
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = text.replace('\xa0', ' ')
    
    return text

In [98]:
def articles_generator(_number, _violation):
    violation = 'nonviolation'
    if _violation: violation = violation[3:]
        
    path = f'hudoc/Article{_number}/{violation}'
    
    os.makedirs(path, exist_ok=True)
    
    for article in articles_json(_number, _violation):
        f = open(f'{path}/{article["id"]}.txt', 'w')
        f.write(article_get(article["id"]))

In [ ]:
articles_generator('3', False)

/home/nti14147326/.local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [80]:
docx_html_url('001-178380')

'https://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-178380'

In [81]:
article_get('001-178380')

/home/nti14147326/.local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


'FIRST SECTION      CASE OF TADIĆ v. CROATIA (Application no. 10633/15)        JUDGMENT    STRASBOURG 23 November 2017 FINAL 09/04/2018 This judgment has become final under Article 44 § 2 of the Convention. It may be subject to editorial revision. In the case of Tadić v. Croatia,The European Court of Human Rights (First Section), sitting as a Chamber composed of:Linos-Alexandre Sicilianos, President,Kristina Pardalos,Krzysztof Wojtyczek,Ksenija Turković,Armen Harutyunyan,Pauliine Koskelo,Jovan Ilievski, judges,and Abel Campos, Section Registrar,Having deliberated in private on 10 October 2017,Delivers the following judgment, which was adopted on that date:PROCEDURE1.\nThe case originated in an application (no. 10633/15) against the Republic of Croatia lodged with the Court under Article 34 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by a Croatian national, Mr Ivica Tadić (“the applicant”), on 23 February 2015.2.\nThe applicant, who h

In [28]:
docx_html_url('001-178380')

'https://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-178380'

In [29]:
article_url(8, False)

'https://hudoc.echr.coe.int/app/query/results?query=contentsitename%3AECHR%20AND%20(NOT%20(doctype%3DPR%20OR%20doctype%3DHFCOMOLD%20OR%20doctype%3DHECOMOLD))%20AND%20((languageisocode%3D%22ENG%22))%20AND%20((documentcollectionid%3D%22GRANDCHAMBER%22)%20OR%20(documentcollectionid%3D%22CHAMBER%22))%20AND%20((nonviolation%3D8))&select=sharepointid,Rank,ECHRRanking,languagenumber,itemid,docname,doctype,application,appno,conclusion,importance,originatingbody,typedescription,kpdate,kpdateAsText,documentcollectionid,documentcollectionid2,languageisocode,extractedappno,isplaceholder,doctypebranch,respondent,ecli,appnoparts,sclappnos&sort=&start=20&length=100&rankingModelId=11111111-0000-0000-0000-000000000000'

In [3]:
r = requests.get("https://hudoc.echr.coe.int/app/query/results?query=contentsitename%3AECHR%20AND%20(NOT%20(doctype%3DPR%20OR%20doctype%3DHFCOMOLD%20OR%20doctype%3DHECOMOLD))%20AND%20((languageisocode%3D%22ENG%22))%20AND%20((documentcollectionid%3D%22GRANDCHAMBER%22)%20OR%20(documentcollectionid%3D%22CHAMBER%22))%20AND%20((nonviolation%3D3))&select=sharepointid,Rank,ECHRRanking,languagenumber,itemid,docname,doctype,application,appno,conclusion,importance,originatingbody,typedescription,kpdate,kpdateAsText,documentcollectionid,documentcollectionid2,languageisocode,extractedappno,isplaceholder,doctypebranch,respondent,ecli,appnoparts,sclappnos&sort=&start=20&length=100&rankingModelId=11111111-0000-0000-0000-000000000000")

In [8]:
json_data = json.loads(r.text)

In [38]:
results = []
for result in json_data['results']:
    results.append({'id': f'{result["columns"]["itemid"]} - {result["columns"]["docname"]}'})

In [39]:
results

[{'id': '001-178380 - CASE OF TADIĆ v. CROATIA'},
 {'id': '001-178698 - CASE OF KITIASHVILI v. GEORGIA'},
 {'id': '001-178962 - CASE OF CEESAY v. AUSTRIA'},
 {'id': '001-178381 - CASE OF HENTSCHEL AND STARK v. GERMANY'},
 {'id': '001-178178 - CASE OF M.F. v. HUNGARY'},
 {'id': '001-177651 - CASE OF DRACI v. THE REPUBLIC OF MOLDOVA AND RUSSIA'},
 {'id': '001-177399 - CASE OF TARJÁNI v. HUNGARY'},
 {'id': '001-177074 - CASE OF KUPARADZE v. GEORGIA'},
 {'id': '001-177295 - CASE OF ZOLOTOREV v. RUSSIA'},
 {'id': '001-176827 - CASE OF MIRZASHVILI v. GEORGIA'},
 {'id': '001-174644 - CASE OF DIMCHO DIMOV v. BULGARIA (No. 2)'},
 {'id': '001-174424 - CASE OF M.O. v. SWITZERLAND'},
 {'id': '001-174652 - CASE OF DAŞLIK v. TURKEY'},
 {'id': '001-173779 - CASE OF DEJNEK v. POLAND'},
 {'id': '001-174223 - CASE OF N.A. v. SWITZERLAND'},
 {'id': '001-173798 - CASE OF APCOV v. THE REPUBLIC OF MOLDOVA AND RUSSIA'},
 {'id': '001-173599 - CASE OF ERIOMENCO v. THE REPUBLIC OF MOLDOVA AND RUSSIA'},
 {'id': 